![train](http://cliparting.com/wp-content/uploads/2016/06/Train-clipart-for-kids-free-free-clipart-images.gif)

In [1]:
import numpy as np
np.random.seed(1019)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import lightgbm

import sklearn
from sklearn.model_selection import train_test_split 

import sys, os, gc, types
import time
from subprocess import check_output

In [2]:
sys.path.append('./utils')

from training import cv, train
from plotting import plot_importance
from data import Data

In [3]:
root_paths = [
    "/data/kaggle-instacart",
    "/Users/jiayou/Dropbox/珺珺的程序/Kaggle/Instacart",
    "/Users/jiayou/Dropbox/Documents/珺珺的程序/Kaggle/Instacart"
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break

# Hyper-Parameter Search

In [4]:
name = 'v12-r0'
down_sample = 1000
test_size = 0.2
num_searches = 1
boosting_rounds = 100
# stopping_rounds = None
aug = False

xgb_params_search = {
#     "learning_rate"    : lambda: int(10**np.random.uniform(-2, -1) * 1e4) / 1e4,
#     "max_depth"        : lambda: np.random.randint(12, 13),
#     "subsample"        : [0.5],
    "tree_method"      : ["hist"],
    "seed"             : list(range(10000))
}

-----------

In [5]:
X_train, X_val, y_train, y_val = Data.train_val(down_sample=down_sample, test_size=test_size, aug=aug)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jiayou/Dropbox/珺珺的程序/Kaggle/Instacart/abt-share/abt_train.sb.orig1.pkl'

In [ ]:
lgb_train = lgb.Dataset(X_train, y_val)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

gc.collect()

In [ ]:
def get_params(default, search):
    np.random.seed(int(time.time()))
    p = dict(default)
    for k, gen in search.items():
        v = None
        if type(gen) == list:
            v = gen[np.random.randint(0, len(gen))]
        elif type(gen) == types.LambdaType:
            v = gen()
        p[k] = v
    return p

def print_params(params, keys):
    print()
    print(["{} = {}".format(k, params[k]) for k in keys])
    print()

In [ ]:
# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
    'num_leaves': 256,
    'min_sum_hessian_in_leaf': 20,
    'max_depth': 12,
    'learning_rate': 0.05,
    'feature_fraction': 0.6,
    # 'bagging_fraction': 0.9,
    # 'bagging_freq': 3,
    'verbose': 1
}

In [ ]:
def lr_schedule(env):
    bst, r = env.model, env.iteration
    if r == 300:
        bst.set_param('learning_rate', 0.05)
    elif r == 500:
        bst.set_param('learning_rate', 0.01)
        
def save_checkpoint(env):
    bst, r = env.model, env.iteration
    if r % 300 == 0:
        bst.save_model(os.path.join(root, 'train-checkpoint.bst'))

In [ ]:
results = []
for i in range(num_searches):
    print('Start training...')
    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=380,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=10)

    print('Save model...')
    # save model to file
    gbm.save_model('model.txt')

    print('Start predicting...')
    # predict
    y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

    print('Feature importances:', list(gbm.feature_importance()))

----